# Import lib

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [ ]:
# Set cố định random seed
np.random.seed(42)
tf.random.set_seed(42)

# Add data

In [ ]:
pos_df = pd.read_csv('review-data/positive_data.csv')
pos_df.head()

In [ ]:
nev_df = pd.read_csv('review-data/nevgative_data.csv')
nev_df.head()

> Đếm số giá trị Pos/ Nev?

In [ ]:
pos_df["label"].value_counts()

In [ ]:
nev_df["label"].value_counts()

> Một số trường hợp class của sample có tỉ lệ quá khác biệt ảnh hưởng đến chất lượng mô hình. Vì vậy nên sử dụng phương thức chọn mẫu `DataFrame.sample()` để cho số lượng mẫu = nhau.

In [ ]:
pos_df_sampled = pos_df.sample(n=len(nev_df), random_state=42) # Lấy mẫu ngẫu nhiên từ pos_df để có số lượng bằng với nev_df

> Kết hợp dữ liệu positive và negative vào cùng 1 DataFrame bằng `pandas.concat`

In [ ]:
df = pandas.concat([pos_df_sampled, nev_df])
df["label"].value_counts()

In [ ]:
# xem du lieu moi nhat
df.tail()

> Để phân phối lại các sample pos/nev trong df, sử dụng phương thức `DataFrame.sample`

> Ở đây chọn `frac=1` để chọn **tất cả** các sample trong df với thứ tự ngẫu nhiên.

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)  # Trộn dữ liệu
df.tail()

> Đầu ra của mô hình phân loại 2 lớp (**binary classification**) là giá trị 0 hoặc 1 => Vì vậy cần `chuyển -1 (nev) -> 0`

In [ ]:
# DataFrame.replace
df['label']= df['label'].replace({-1:0})
df['label'].value_counts()

# Chia tập train:test
- Sử dụng thuộc tính `values` của Pandas series để lấy các giá trị dưới dạng **numpy ndarray**
- Các dữ liệu dung để train và test nên được chuyển sang `ndarray` để phù hợp với các hàm tính toán trong ML và DL. 

In [ ]:
X = df['Review'].values
y = df['Label'].values
X.shape, y.shape

> Sử dụng hàm `train_test_split` để chia tập train:test **(tỉ lệ 8:2)**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# doi kieu du lieu ve numpy array
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Tokenize dữ liệu train - test
Vận dụng các kiến thức tokenizing và padding ở buổi trước để xử lý dữ liệu chữ.\
Nhắc lại các bước xử lý:
1. Chọn kích thước từ điển
2. Chọn độ dài lớn nhất của một chuỗi (sequence)
3. Tạo Tokenizer và fit Tokenizer (trên văn bản của tập train)
4. Sử dụng Tokenizer đã huấn luyện để tạo tokens
5. Sử dụng padding và truncating để các chuỗi có độ dài bằng nhau

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

In [ ]:
# chọn kích thước từ điển là 10000
# độ dài lớn nhất của 1 bình luận 400 tokens
vocab_size = 10000
max_length = 400

In [ ]:
# Tạo tokenizer và fit
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train) # đánh so
# sử dụng tokenizer để train trên tập train
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post') # đệm câu nếu thiếu/ cắt câu nếu dư

In [ ]:
# Sử dụng tokenizer để fit trên tập test
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post', truncating='post')

In [ ]:
print("Length of train dataset:", len(X_train_pad))
print("Length of test dataset:", len(X_test_pad))